In [23]:
%pip install python-dotenv matplotlib pandas -q



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import requests
import json
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

def fetch_air_quality(city):
    token = os.getenv("API_TOKEN")
    url = f"https://api.waqi.info/feed/{city}/?token={token}"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.text)
        if data['status'] == 'ok':
            return data['data']
        else:
            return f"Error en la API: {data['status']}"
    elif response.status_code == 400:
        return "Solicitud no válida"
    elif response.status_code == 401:
        return "Token inválido"
    else:
        return f"Error: {response.status_code}"

if __name__ == "__main__":
    cities = ["amsterdam", "berlin", "paris"]

    for city in cities:
        air_quality_data = fetch_air_quality(city)
        print(f"Datos de calidad del aire para {city.capitalize()}:")
        print(air_quality_data)
        print("-" * 40)

Datos de calidad del aire para Amsterdam:
{'aqi': 14, 'idx': 5771, 'attributions': [{'url': 'http://www.luchtmeetnet.nl/', 'name': 'RIVM - Rijksinstituut voor Volksgezondheid en Milieum, Landelijk Meetnet Luchtkwaliteit', 'logo': 'Netherland-RIVM.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [52.3702157, 4.8951679], 'name': 'Amsterdam', 'url': 'https://aqicn.org/city/amsterdam', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'co': {'v': 3.5}, 'h': {'v': 67.2}, 'no2': {'v': 10.4}, 'p': {'v': 1015.4}, 'pm10': {'v': 8}, 'pm25': {'v': 14}, 'so2': {'v': 0.3}, 't': {'v': 9.5}, 'w': {'v': 4}}, 'time': {'s': '2023-10-17 20:00:00', 'tz': '+02:00', 'v': 1697572800, 'iso': '2023-10-17T20:00:00+02:00'}, 'forecast': {'daily': {'o3': [{'avg': 17, 'day': '2023-10-17', 'max': 28, 'min': 9}, {'avg': 17, 'day': '2023-10-18', 'max': 22, 'min': 15}, {'avg': 13, 'day': '2023-10-19', 'max': 18, 'min': 9}, {'avg': 18, 'day': '2023-10-20', 'max': 24, 'min'

In [18]:
import os
import json
import requests
import pandas as pd

def fetch_and_convert_to_dataframe(cities):
    token = os.getenv("API_TOKEN")
    dfs = []  # Lista para guardar los DataFrames

    for city in cities:
        url = f"https://api.waqi.info/feed/{city}/?token={token}"
        response = requests.get(url)
        data = None

        if response.status_code == 200:
            data = json.loads(response.text)
            if data['status'] != 'ok':
                print(f"Error en la API para la ciudad {city}: {data['status']}")
                continue
        else:
            print(f"Error en la solicitud para la ciudad {city}: {response.status_code}")
            continue

        # Aplanar el diccionario JSON y convertirlo en un DataFrame de pandas
        df = pd.json_normalize(data['data'])
        dfs.append(df)  # Agregar el DataFrame a la lista

    # Concatenar todos los DataFrames en uno solo
    if dfs:
        final_df = pd.concat(dfs, keys=cities, axis=0).reset_index().drop('level_1', axis=1)
        final_df.rename(columns={'level_0': 'City'}, inplace=True)
        return final_df
    else:
        return "No se pudo obtener ningún DataFrame"

if __name__ == "__main__":
    cities = ["amsterdam", "berlin", "paris"]
    final_df = fetch_and_convert_to_dataframe(cities)


In [19]:
final_df

,City,aqi,idx,attributions,dominentpol,city.geo,city.name,city.url,city.location,iaqi.co.v,...,time.s,time.tz,time.v,time.iso,forecast.daily.o3,forecast.daily.pm10,forecast.daily.pm25,forecast.daily.uvi,debug.sync,iaqi.o3.v
0,amsterdam,14,5771,"[{'url': 'http://www.luchtmeetnet.nl/', 'name'...",pm25,"[52.3702157, 4.8951679]",Amsterdam,https://aqicn.org/city/amsterdam,,3.5,...,2023-10-17 20:00:00,+02:00,1697572800,2023-10-17T20:00:00+02:00,"[{'avg': 17, 'day': '2023-10-17', 'max': 28, '...","[{'avg': 13, 'day': '2023-10-17', 'max': 19, '...","[{'avg': 36, 'day': '2023-10-17', 'max': 51, '...","[{'avg': 0, 'day': '2022-10-24', 'max': 0, 'mi...",2023-10-18T03:30:32+09:00,NaN
1,berlin,34,6132,[{'url': 'http://www.stadtentwicklung.berlin.d...,pm25,"[52.5200066, 13.404954]","Berlin, Germany",https://aqicn.org/city/germany/berlin,,0.1,...,2023-10-17 20:00:00,+02:00,1697572800,2023-10-17T20:00:00+02:00,"[{'avg': 24, 'day': '2023-10-15', 'max': 29, '...","[{'avg': 7, 'day': '2023-10-15', 'max': 9, 'mi...","[{'avg': 15, 'day': '2023-10-15', 'max': 17, '...","[{'avg': 1, 'day': '2021-07-03', 'max': 4, 'mi...",2023-10-18T03:58:36+09:00,2.9
2,paris,55,5722,"[{'url': 'https://www.airparif.asso.fr/', 'nam...",pm25,"[48.856614, 2.3522219]",Paris,https://aqicn.org/city/paris,,0.1,...,2023-10-17 17:00:00,+02:00,1697562000,2023-10-17T17:00:00+02:00,"[{'avg': 13, 'day': '2023-10-17', 'max': 22, '...","[{'avg': 15, 'day': '2023-10-17', 'max': 19, '...","[{'avg': 52, 'day': '2023-10-17', 'max': 60, '...","[{'avg': 0, 'day': '2021-02-26', 'max': 1, 'mi...",2023-10-18T03:48:09+09:00,10.0
